In [1]:
from __future__ import print_function
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [27]:
def load_clean_data(csv_file):
    df = pd.read_csv(csv_file, header = 0)
    df['Age'] = df['Age'].fillna(df.Age.median())
    df['Gender'] = df['Sex'].map({'female':0, 'male':1}).astype(int)
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    df['Embarked'] = df['Embarked'].fillna('S')
    df = df.drop(['SibSp','Parch','Name','Sex','Ticket','Cabin','Embarked'],axis=1)
    # One hot encoding for "Pclass"
    df = df.join(pd.get_dummies(df['Pclass'], prefix ='Class').astype(int))
    df = df.drop(['Pclass'], axis = 1)
    return df
train_df = load_clean_data('train.csv')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Age            891 non-null float64
Fare           891 non-null float64
Gender         891 non-null int64
Class_1        891 non-null int64
Class_2        891 non-null int64
Class_3        891 non-null int64
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


In [17]:
test_df = load_clean_data('test.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
PassengerId    418 non-null int64
Age            418 non-null float64
Fare           418 non-null float64
Gender         418 non-null int64
Class_1        418 non-null int64
Class_2        418 non-null int64
Class_3        418 non-null int64
dtypes: float64(2), int64(5)
memory usage: 22.9 KB


In [28]:
x_data = train_df.drop(['PassengerId', "Survived"], axis = 1)
y_data = train_df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

In [29]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train,y_train)
random_forest.score(x_test,y_test)

0.81005586592178769

In [22]:
test_data = test_df.drop(['PassengerId'], axis = 1)
y_pred = random_forest.predict(test_data)
submission = pd.DataFrame({
        "PassengerId":test_df['PassengerId'],
        "Survived":y_pred
    })
submission.to_csv('titanic.csv',index=False)